In [135]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
def get_soup(url):
    """Function to get the soup object from a given URL."""
    response = requests.get(url)
    return BeautifulSoup(response.text, 'html.parser')

def extract_zip_links(url):
    """Function to find all .zip links on a given webpage."""
    soup = get_soup(url)
    return [urljoin(url, link.get('href')) for link in soup.find_all('a') if link.get('href') and link.get('href').endswith('.zip')]

base_url = "https://static.case.law/"
soup = get_soup(base_url)

# Extract all hrefs from the base URL
hrefs = [urljoin(base_url, link.get('href')) for link in soup.find_all('a') if link.get('href')]



# Initialize list to store all zip links
all_zip_links = []

count = 0
for href in hrefs[:1]:
    soup = get_soup(href)
    urls = [a['href'] for a in soup.find_all('a', href=True) if re.search(r'/\d+/$', a['href'])]
    for url in urls[:1]:
        soup = get_soup(url+"html/")
        print(soup.find_all('a'))
        links = [link for link in soup.find_all('a') if link.get('href') and link.get('href').endswith('.html')]
        print(links)


[<a href="https://static.case.law/">Home</a>, <a href="https://static.case.law/a2d/">a2d</a>, <a href="https://static.case.law/a2d/31/">31</a>, <a href="https://static.case.law/a2d/31/html/0647-01.html">0647-01.html</a>, <a href="https://static.case.law/a2d/31/html/0649-01.html">0649-01.html</a>, <a href="https://static.case.law/a2d/31/html/0652-01.html">0652-01.html</a>, <a href="https://static.case.law/a2d/31/html/0654-01.html">0654-01.html</a>, <a href="https://static.case.law/a2d/31/html/0655-01.html">0655-01.html</a>, <a href="https://static.case.law/a2d/31/html/0657-01.html">0657-01.html</a>, <a href="https://static.case.law/a2d/31/html/0657-02.html">0657-02.html</a>, <a href="https://static.case.law/a2d/31/html/0659-01.html">0659-01.html</a>, <a href="https://static.case.law/a2d/31/html/0660-01.html">0660-01.html</a>, <a href="https://static.case.law/a2d/31/html/0662-01.html">0662-01.html</a>, <a href="https://static.case.law/a2d/31/html/0669-01.html">0669-01.html</a>, <a href="

In [127]:
soup.html

In [11]:
import zipfile
import os

# Directory setup
zip_input_dir = '../../../lawyer_data/zip/'  # Change this to the path where your zip files are stored
unzip_dir = '../../../lawyer_data/unzip'  # Change this to your preferred unzip directory
html_dir = '../../../lawyer_data'  # Change this to your preferred HTML storage directory

os.makedirs(unzip_dir, exist_ok=True)
os.makedirs(html_dir, exist_ok=True)

# Process zip files from the specified directory
for zip_filename in os.listdir(zip_input_dir):
    if zip_filename.endswith('.zip'):
        zip_path = os.path.join(zip_input_dir, zip_filename)
        
        # Unzipping process
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)
            print(f'Extracted {zip_filename} in {unzip_dir}')

            # Move HTML files to the html directory
            for file in zip_ref.namelist():
                if file.endswith('.html'):
                    source_file = os.path.join(unzip_dir, file)
                    destination_file = os.path.join(html_dir, file)
                    os.rename(source_file, destination_file)
                    print(f'Moved {file} to {html_dir}')


Extracted a2d34.zip in ../../../lawyer_data/unzip
Moved html/0026-01.html to ../../../lawyer_data
Moved html/0027-01.html to ../../../lawyer_data
Moved html/0029-01.html to ../../../lawyer_data
Moved html/0031-01.html to ../../../lawyer_data
Moved html/0033-01.html to ../../../lawyer_data
Moved html/0035-01.html to ../../../lawyer_data
Moved html/0036-01.html to ../../../lawyer_data
Moved html/0039-01.html to ../../../lawyer_data
Moved html/0041-01.html to ../../../lawyer_data
Moved html/0124-01.html to ../../../lawyer_data
Moved html/0126-01.html to ../../../lawyer_data
Moved html/0257-01.html to ../../../lawyer_data
Moved html/0259-01.html to ../../../lawyer_data
Moved html/0261-01.html to ../../../lawyer_data
Moved html/0353-01.html to ../../../lawyer_data
Moved html/0358-01.html to ../../../lawyer_data
Moved html/0359-01.html to ../../../lawyer_data
Moved html/0468-01.html to ../../../lawyer_data
Moved html/0608-01.html to ../../../lawyer_data
Moved html/0609-01.html to ../../../la

In [43]:
from langchain_community.document_loaders import UnstructuredHTMLLoader
urls = '../../../lawyer_data/html'
documents = []
for url in os.listdir(urls)[:20]:
    url = urls+"/"+url
    print(url)
    loader = UnstructuredHTMLLoader(url)
    document = loader.load()
    text = document[0].page_content
    documents.append(text)
    

../../../lawyer_data/html/0298-01.html
../../../lawyer_data/html/0867-01.html
../../../lawyer_data/html/0096-01.html
../../../lawyer_data/html/0877-01.html
../../../lawyer_data/html/0712-01.html
../../../lawyer_data/html/0359-01.html
../../../lawyer_data/html/0349-01.html
../../../lawyer_data/html/0231-01.html
../../../lawyer_data/html/0243-01.html
../../../lawyer_data/html/0253-01.html
../../../lawyer_data/html/0608-01.html
../../../lawyer_data/html/0618-01.html
../../../lawyer_data/html/0760-01.html
../../../lawyer_data/html/0125-01.html
../../../lawyer_data/html/0135-01.html
../../../lawyer_data/html/0805-01.html
../../../lawyer_data/html/0815-01.html
../../../lawyer_data/html/0382-01.html
../../../lawyer_data/html/0392-01.html
../../../lawyer_data/html/0178-01.html


In [114]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

from dotenv import load_dotenv
from os import getenv
load_dotenv("../../../.env")
my_key = getenv("OPENAI_API_KEY")

text_splitter = CharacterTextSplitter(chunk_size=10000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)
db = Chroma.from_documents(documents, OpenAIEmbeddings(api_key=my_key))

In [115]:
query = "What did the president say about School v Tibbs"
docs = db.similarity_search(query)
docs

[Document(page_content="But I do not agree that while we are waiting for Congress to act the courts must sit supinely by, and protest our helplessness to construe the existing statute so as to relieve against oppression. We as judges cannot change the statute. “But,” as the Court said in the Clark case, supra, “if we are in one of the ‘open spaces’ in the law of this jurisdiction we must fill it as well as we can, with a view to the social interests which seem to be involved and with such aid as we can get from authorities, elsewhere and from ‘logic, and history, and' custom, and utility, and the accepted standards of right conduct.’ We cannot evade this duty; for unless we establish a right in the plaintiff we establish a privilege or immunity in the defendant.”\n\nIt is against the unwarranted immunity created by the decision of the majority that I protest.", metadata={'source': '../../../lawyer_data/html/0178-01.html'}),
 Document(page_content='Even if the judge in the second case t

In [77]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [69]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.8)
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=db.as_retriever())

chain(query)


{'query': 'What did the president say about Ketanji Brown Jackson',
 'result': "I don't have any information about what the president said about Ketanji Brown Jackson."}

In [116]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

retriever = db.as_retriever()

# Function to format the documents into a single string
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Initialize the language model from OpenAI with specific parameters
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7, max_tokens=2000)


from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{context} {question}")
])



# Configure the Rag (Retrieval-Augmented Generation) Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



# Function to create a legal briefing with a specified word count
def create_legal_briefing(case_name):
    prompt = f"Create a detailed legal briefing about {case_name}. Please ensure the briefing is comprehensive and approximately 1000 words in length."
    return rag_chain.invoke(prompt)

# Invoke the chain to generate a legal briefing
legal_briefing = create_legal_briefing("Julia Susser")
print(legal_briefing)


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 24884 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [84]:
llm.invoke("Write a legal briefing about Julia Susser")

AIMessage(content="Legal Briefing on Julia Susser\n\nI. Introduction\nJulia Susser is a 35-year-old woman who has recently been charged with embezzlement from her employer, XYZ Corporation. The prosecution alleges that Susser used her position as a financial analyst to steal funds from the company over the course of several months. This legal briefing will examine the evidence against Susser and provide an analysis of the potential legal implications of the case.\n\nII. Evidence Against Susser\nThe prosecution has presented several pieces of evidence to support their case against Susser. First, they have financial records showing a series of unauthorized transactions made by Susser using company funds. These transactions include transfers to Susser's personal bank account and payments to various vendors that have no connection to XYZ Corporation.\n\nAdditionally, the prosecution has obtained emails and documents from Susser's work computer that suggest she was aware of her actions and 

In [82]:

prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a world class technical documentation writer.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='{context} {question}'))])

In [62]:
pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


In [110]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://static.case.law/cal-rptr-3d/196/html/0001-01.html",),
)
docs = loader.load()

In [111]:
documents = [docs[0]]

'\n\nTiffany BRINKLEY, Plaintiff and Appellant, v. MONTEREY FINANCIAL SERVICES, INC., Defendant and Respondent.\nD066059\nCourt of Appeal, Fourth District, Division 1, California.\nFiled 11/19/2015\nNiddrie Fish & Addams, Niddrie Addams, David A. Niddrie, John S. Addams, San Diego; Keegan & Baker, Patrick N. Keegan, Carlsbad; Wickman & Wickman, Steven A. Wickman and Christina E. Wickman, Escondido, for Plaintiff and Appellant.\nCall & Jensen, Matthew R. Orr and Melinda Evans, Newport Beach, for Defendant and Respondent.\n\n\nAARON, J.\n*321I.\nINTRODUCTION\nPlaintiff Tiffany Brinkley appeals from an order of the trial court compelling her to arbitrate her individual claims and dismissing her class claims. Brinkley filed a putative class action against defendant Monterey Financial Services, Inc. (Monterey), asserting statutory violations arising from allegations that Monterey unlawfully recorded and/or monitored telephone conversations that Brinkley had with Monterey\'s representatives.